# Assignment 7: Solving Problems with Local Search 

Eider Diaz A00828174, Campus Monterrey

Carlos Hinojosa A01137566, Campus Monterrey

Miguel Cortes A01270966, Campus Monterrey

## Set-Covering Problem 

In a set-covering model, each member of a given set (set 1) must be “covered” by an acceptable member of another set (set 2). The objective in a set-covering problem is to minimize the number of elements in set 2 that are needed to cover all of the elements in set 1. 

### Simulated Annealing

<b>i) How do you describe and codify a solution and how will you represent it in Python for the chosen method?</b>
To codify the solution to the Set-Covering Problem using Simulated Annealing, we use the code provided by AIMA. We stated this problem as an optimization problem, where we would try to minimize the number of cameras used from the initial set of twelve.

<b>ii) How do you generate the initial solution(s)?</b>
The initial solution was formed by using all 12 cameras to cover the 25 zones.

<b>iii) How are the neighbors (or offspring) of the current solution(s) generated during the execution of the method?</b>
For each state, neighbors were generated by a couple methods. The first method for generating neighbors was deleting each one of the existing cameras, through this we obtained N neighbors (where n is the total cameras in the current state of the solution). The other method was, where possible, add one of the unused cameras to the solution, through which we obtained additional M neighbors (where M is 12 - N).

<b>iv) How do you calculate the objective function (or fitness) you want to optimize?</b>
The objective function was calculated in two steps. First, we check for the feasibility of the solution. For a solution to be feasible, we considered that it must cover all 25 zones, zones 1 and 2 must be covered by at least two cameras and camera 7 must be part of the solution.

Once clearing that the solution was feasible, we considered the number of cameras used as the number we wanted to minimize.


In [7]:
from search import *

In [8]:
class Set_covering(Problem):
    '''Initial state should be a naive solution to the problem,
    that is a list containing all the subsets of the universe that
    are given for minimization. This initial state is a solution
    because the union of these subsets equals the universe (it is 
    the worst solution possible because we are using them all), but
    better solutions will be searched from this initial state. The goal
    state is the universe and we will use it to check if a given collection
    of subsets covers the universe upon union'''
    
    def actions(self, state):
        '''For each state, the possible actions to generate successors
        are to remove subsets (if the collection of subsets is not empty)
        or to add subsets (if the collection of subsets is not the initial state)'''
        actions = []
        
        #Generate neighbors by removing each element in the existing solution
        for element in state:
            actions.append( ("remove", element) )
        
        #Generate additional neighbors by including each unused element to the existing solution
        notpresent = set(self.initial) - set(state)
        for element in notpresent:
            actions.append( ("add", element) )
            
        return actions
    
    def result(self, state, action):
        result = state.copy()
        if action[0] == "remove":
            result.remove(action[1])
        elif action[0] == "add":
            result.append(action[1])
        return result
    
    def value(self, state):
        '''Negative values returned as this is a minimization problem'''
        covered = set()
        ones = twos = 0
        camera7 = False
        #Check for constraints: covering twice zones 1 and 2, using camera 7
        for subset in state:
            if subset == c7:
                camera7 = True
            for element in subset.coverage:
                if element == 1:
                    ones += 1
                elif element == 2:
                    twos += 1
                covered.add(element)
                
        if covered != self.goal: #This collection of subsets is invalid, treat it as worse than initial state
            return -len(self.initial) - 1
        else:
            if ones >= 2 and twos >= 2 and camera7 == True: #Cover twice zones 1 and 2, uses camera 7
                return -len(state)
            else:
                return -len(self.initial) - 1 #invalid state too

In [9]:
#Define the camera locations and their coverage as specified in the assignment
class Camera:
    def __init__(self, name, coverage):
        self.name = name
        self.coverage = coverage
        
    def __repr__(self):
        return self.name

c1 = Camera("c1", {1, 3, 4, 6, 7})
c2 = Camera("c2", {4, 7, 8, 12})
c3 = Camera("c3", {2, 5, 9, 11, 13})
c4 = Camera("c4", {1, 2, 18, 19, 21})
c5 = Camera("c5", {3, 6, 10, 12, 14})
c6 = Camera("c6", {8, 14, 15, 16, 17})
c7 = Camera("c7", {18, 21, 24, 25})
c8 = Camera("c8", {2, 10, 16, 23})
c9 = Camera("c9", {1, 6, 11})
c10 = Camera("c10", {20, 22, 24, 25})
c11 = Camera("c11", {2, 4, 6, 8})
c12 = Camera("c12", {1, 6, 12, 17})

cameras = [c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12]

#Define the universe set by union of all subsets (set with all stadium locations covered)
stadium = set()
for camera in cameras:
    stadium.update(camera.coverage)

# Results and Experiments

In [34]:
#Solve the CBS cameras problems with simulated annealing
CBS_cameras = Set_covering(initial = cameras.copy(), goal = stadium)

#We need to try different parameters for the exp_schedule function
solution = simulated_annealing(CBS_cameras, exp_schedule(k=20, limit=850))
print(solution.state) #Set of cameras found by simulated annealing

#Verify that the solution covers all the stadium locations
covered=set()
for camera in solution.state:
    covered.update(camera.coverage)
if len(covered) == 25:
    print("All stadium locations covered with " + str(len(solution.state)) + " cameras")
else:
    print("Solution is invalid, not all locations are covered")

[c8, c2, c10, c4, c3, c7, c6, c1]
All stadium locations covered with 8 cameras


So far we've observed that in order to get valid solutions we had to increase the limit of the schedule function from the default (100) to 1000, this implies we need to choose a correct limit in order to get a good solution.
When we run the algorithm with a limit of temperature = 1000 on the schedule function we get always get the optimal solution which is  
### [c8, c2, c10, c4, c3, c7, c6, c1]
This is the <b>optimal solution</b> we found, where all stadium locations covered with <b>8 cameras</b>

On the other hand we have test empirically with binary search to find the lowest value for this limit. We found that using 825 or less as the limit, we have get stuck on local optimas solutions like:
### [c4, c6, c1, c2, c3, c5, c8, c7, c11, c10] 
Using limit 825 we get a suboptimal solution using 10 cameras to satisfy every constraint.

We also ran several experiments with different values on the parameters, with no different results. These experiments are summarized in the following Table.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg" style="undefined;table-layout: fixed; width: 349px">
<colgroup>
<col style="width: 52px">
<col style="width: 44px">
<col style="width: 252.5px">
</colgroup>
  <tr>
    <th class="tg-7btt">K</th>
    <th class="tg-7btt">Limit</th>
    <th class="tg-7btt">Solution</th>
  </tr>
  <tr>
    <td class="tg-0pky">20</td>
    <td class="tg-0pky">100</td>
    <td class="tg-0pky">No Solution</td>
  </tr>
  <tr>
    <td class="tg-0pky">20<br></td>
    <td class="tg-0pky">1000</td>
    <td class="tg-0pky">[c8, c2, c10, c4, c3, c7, c6, c1]</td>
  </tr>
  <tr>
    <td class="tg-0pky">20<br></td>
    <td class="tg-0pky">500</td>
    <td class="tg-0pky">No Solution</td>
  </tr>
  <tr>
    <td class="tg-0pky">20</td>
    <td class="tg-0pky">750</td>
    <td class="tg-0pky">No Solution<br></td>
  </tr>
  <tr>
    <td class="tg-0lax">20</td>
    <td class="tg-0lax">875</td>
    <td class="tg-0lax">[c8, c2, c10, c4, c3, c7, c6, c1]</td>
  </tr>
  <tr>
    <td class="tg-0lax">20</td>
    <td class="tg-0lax">813</td>
    <td class="tg-0lax">No Solution</td>
  </tr>
  <tr>
    <td class="tg-0lax">20</td>
    <td class="tg-0lax">825</td>
    <td class="tg-0lax">[c4, c6, c1, c2, c3, c5, c8, c7, c11, c10]<br></td>
  </tr>
  <tr>
    <td class="tg-0lax">100</td>
    <td class="tg-0lax">1000<br></td>
    <td class="tg-0lax"><span style="font-weight:400;font-style:normal;text-decoration:none">[c8, c2, c10, c4, c3, c7, c6, c1]</span></td>
  </tr>
  <tr>
    <td class="tg-0lax">1000</td>
    <td class="tg-0lax">1000</td>
    <td class="tg-0lax"><span style="font-weight:400;font-style:normal;text-decoration:none">[c8, c2, c10, c4, c3, c7, c6, c1]</span></td>
  </tr>
  <tr>
    <td class="tg-0lax">10000<br></td>
    <td class="tg-0lax">1000</td>
    <td class="tg-0lax"><span style="font-weight:400;font-style:normal;text-decoration:none">[c8, c2, c10, c4, c3, c7, c6, c1]</span><br></td>
  </tr>
</table>

Furthermore just to test out if the constraints described in the assignment make the solution way different we run the algorithm without the constraints condition and that generate the following solutions
### [c3, c5, c6, c8, c4, c10, c2]
Global Solution covering every zone, but <b>not satisfying the constraints of using camera 7 and covering twice the zones 1 and 2</b>.